In [1]:
import tensorflow as tf
from tensorflow import keras
#print(tf.keras.__version__)
#print(tf.__version__)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math as mt
import random as rng

from sklearn import preprocessing

def getColumns(size):
    columns = []
    for i in range(size):
        columns.append(i)
    return columns

directory = "./CMAPSSData/train_FD001.txt"
turbo = pd.read_csv(directory, header=None, sep=' ')
turbo = turbo.drop([26, 27], axis=1)

#for i in turbo.columns:
#    column = turbo[i]
#    print(i)
#    print(column.describe())
    
#print(turbo.values)

# Drop the input parameters to get just the sensors
turboSens = turbo.drop([0,1], axis=1)
turboSens.columns = getColumns(turboSens.shape[1])

# Get the cycles and useful sensors. 
turboUsefulSens = turboSens

min_max_scaler = preprocessing.MinMaxScaler()
norm_sens = min_max_scaler.fit_transform(turboUsefulSens)

turboCycles = turbo[1]
#print(turboCycles)

deteriorate = []
maxCycle = float(turboCycles.max())

currentCount = maxCycle
for i in range(turbo.shape[0] - 1, -1, -1):
    currentRow = turboCycles.loc[i]
    deteriorate.insert(0, currentCount)
    if currentRow == 1:
        currentCount = maxCycle
    else:
        currentCount -= 1

        
deteriorate = pd.DataFrame(deteriorate)

deteriorateSwap = []
for i in deteriorate[0]:
    deteriorateSwap.append(maxCycle - i)
deteriorate = pd.DataFrame(deteriorateSwap)


norm_sens = pd.DataFrame(norm_sens)
deteriorate = deteriorate.join(norm_sens, lsuffix='0', rsuffix='1')
deteriorate.columns = getColumns(deteriorate.shape[1])



In [2]:
array = deteriorate.to_numpy()

arrayCopy = array.tolist()

simulations = []

convCount = 10
convDiv = int(convCount/2)

count = len(array)

while(count > 0):
    cycles = []
    element = [-1,-1]
    while(element[0]!=0):
        element = arrayCopy.pop(0)
        cycles.append(element)
        count-=1
    simulations.append(cycles)
    
#for sim in simulations:
#    lenth = len(sim)
#    remainder = lenth%convCount
#    for i in range(remainder):
#        sim.pop(0)
    
#simsRecomb = []
#for sim in simulations:
#    for i in sim:
#        simsRecomb.append(i)

#print(len(simsRecomb))

In [3]:

tens = []
tensCount = []

for sim in simulations:
    for i in range(convDiv, len(sim) - convDiv):
        ten = []
        total = 0
        for j in range(-convDiv,convDiv):
            index = i + j
            element = sim[index].copy()
            total+= element.pop(0)
            ten.append(element)
        total/=convCount
        tensCount.append(total)
        tens.append(ten)

    
print(len(tens))
print(len(tensCount))

19631
19631


In [4]:
simsReorder = []
simsCycleReorder = []

length = len(tens)
for i in range(length):
    index = rng.randint(0, len(tens)-1)
    simsReorder.append(tens.pop(index))
    simsCycleReorder.append(tensCount.pop(index))
    
    
    
simsTrain = simsReorder[:15704]
targetTrain = simsCycleReorder[:15704]

simsTest = simsReorder[15704:]
targetTest = simsCycleReorder[15704:]



width = (turboUsefulSens.shape[1])
print(len(simsTrain[0][0]))

24


In [10]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(150, 3, activation='relu', input_shape=(convCount, width)),
    tf.keras.layers.Conv1D(300, 3, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.MaxPooling1D(6),
    tf.keras.layers.LSTM(300, return_sequences=True, dropout=0.15, recurrent_dropout=0.15),
    tf.keras.layers.LSTM(400, dropout=0.15, recurrent_dropout=0.15),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1)
])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 8, 150)            10950     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 6, 300)            135300    
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 300)            0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 300)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 1, 300)            721200    
_________________________________________________________________
lstm_5 (LSTM)                (None, 400)               1121600   
_________________________________________________________________
dense_4 (Dense)              (None, 100)              

In [7]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='mse', 
              metrics=['mae', 'mse'])

In [9]:
model.fit(simsTrain, targetTrain, epochs=500, verbose=1, validation_data=(simsTest,targetTest))

Train on 15704 samples, validate on 3927 samples
Epoch 1/500
15704/15704 [==============================] - 22s 1ms/sample - loss: 127.8485 - mae: 7.6021 - mse: 127.8485 - val_loss: 186.1679 - val_mae: 8.6957 - val_mse: 186.1679
Epoch 2/500
15704/15704 [==============================] - 21s 1ms/sample - loss: 134.0935 - mae: 7.7305 - mse: 134.0936 - val_loss: 190.7431 - val_mae: 8.7175 - val_mse: 190.7430
Epoch 3/500
15704/15704 [==============================] - 19s 1ms/sample - loss: 133.8280 - mae: 7.6515 - mse: 133.8280 - val_loss: 199.5897 - val_mae: 8.9661 - val_mse: 199.5897
Epoch 4/500
15704/15704 [==============================] - 20s 1ms/sample - loss: 121.3706 - mae: 7.4765 - mse: 121.3707 - val_loss: 192.9741 - val_mae: 9.0907 - val_mse: 192.9741
Epoch 5/500
15704/15704 [==============================] - 21s 1ms/sample - loss: 123.4847 - mae: 7.5037 - mse: 123.4848 - val_loss: 211.1814 - val_mae: 9.1823 - val_mse: 211.1815
Epoch 6/500
15704/15704 [==========================

KeyboardInterrupt: 

In [ ]:
model.evaluate(simsTest,targetTest)

In [9]:


print(model.predict([simsTest[0]]))
 
print(targetTest[0])

print(model.predict([simsTest[1]]))

print(targetTest[1])

[[[309.90103]]]
295.5
[[[17.469666]]]
16.5
